<p><strong><font size="6">WalOUS project</font></strong></p>

<p><strong><font size="6">C_2_Classification_Rule_based</font></strong></p>

WALOUS_UTS - Copyright (C) <2020> <Université catholique de Louvain (UCLouvain), Belgique;
					 Université Libre de Bruxelles (ULB), Belgique;
					 Institut Scientifique de Service Public (ISSeP), Belgique;
					 Service Public de Wallonie (SWP), Belgique >
						 							
	
List of the contributors to the development of WALOUS_UTS: see LICENSE file.


Description and complete License: see LICENSE file.
	
This program (WALOUS_UTS) is free software:
you can redistribute it and/or modify it under the terms of the
GNU General Public License as published by the Free Software
Foundation, either version 3 of the License, or (at your option)
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program (see COPYING file).  If not,
see <http://www.gnu.org/licenses/>.

-------
Jupyter Notebook containing the preprocessing steps consisting of: 
- Defining the order for the hierarchical rules for the classification.
- Performing the rule-based classification. 

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

# Define working environment

**Import libraries**

In [ ]:
# Import libraries needed for setting parameters of operating system 
import os
import sys

**Add folder with SCR provided belong to this notebook**

In [ ]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

**Setup environment variables**

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [ ]:
run ../SRC/config.py

In [ ]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [ ]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

**Other functions**

In [ ]:
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that check and create folder
from mkdir import check_create_dir

**Custom functions: Psycopg2 and Postgresql functions**

In [ ]:
# Import function that display postgresql's table header
from display_header import display_header
# Import function to creation connection to Postgresql database 
from postgres_functions import create_pg_connexion
# Import function to create index
from postgres_functions import create_index

In [ ]:
# Import function to create generate decision-tree query and produce result table
from postgres_classification import decision_tree_classification

In [ ]:
# Import function to export postgis table as shapefile or geopackage
from postgres_export import ogr2ogr_export

In [ ]:
# Import functions for creation of postgresql back-up and restoration
from dump_functions import dump_table, dump_db, restore_psql

# Classification of land use using decision-tree

## Define list of nested rules for walous_maj

In [ ]:
# L'ensemble des règles de classification sont appliqués de haut en bas. Si la condition d'une règle 
# supérieure est rencontrée pour une parcelle donnée, elle se voit attribuer sa classe par cette règle. 
# Exemple 1 : Une parcelle cadastrale classée à la règle 1 ne sera plus impactée par les règles sous-jacentes. 
# Exemple 2 : Une parcelle classée à la règle 18 n'aura remplis aucune des conditions des règles précédentes.
list_rules = []

# 1 - Si la parcelle cadastrale n'est caractérisée par aucune base de données, 
# attribuer la classe "usage inconnu cadastré"
list_rules.append(("all_hilucs is NULL","'6_6_A'"))

# 2 - Si la parcelle cadastrale est caractérisée par un et un seul élément de valeur '8_8', indiquant que 
# l'utilisation est incertaine selon la table de correspondance de la 'Nature cadastrale' vers HILUCS, 
# attribuer la classe "usage inconnu cadastré"
list_rules.append(("Cardinality(all_hilucs) = 1 AND all_hilucs[1] = '8_8'","'6_6_A'"))

# 3 - Si la parcelle cadastrale est caractérisée par un et un seul élément, attribuer la classe correspondante
list_rules.append(("Cardinality(all_hilucs) = 1","all_hilucs[1]"))

# 4 - Si la parcelle cadastrale est caractérisée par au moins deux bases de données fournissant toutes 
# la même classe, attribuer la classe correspondante
list_rules.append(("agreement_all_hilucs = True","all_hilucs[1]"))

# 5 - Si la parcelle cadastrale est caractérisée par exactement deux bases de données attribuant toutes 
# les deux la classe 'usage résidentiel' ('5') au niveau 1 de HILUCS, et pour laquelle le cadastre 
# précise qu'il s'agit d'un 'usage résidentiel permanent', attribuer cette classe
list_rules.append(("Cardinality(all_hilucs) = 2 AND agreement_l1_hilucs = True AND l2_hilucs[1] = '5_1'",
                   "l2_hilucs[1]"))

# 6 - Si la parcelle cadastrale est caractérisée par plus d'une base de données, que l'ensemble des bases 
# de données sont en accord au niveau 2 de HILUCS et qu'une seule base de données fournis une 
# information au niveau 3 de HILUCS, attribuer cette classe
list_rules.append(("Cardinality(all_hilucs) > 1 AND agreement_l2_hilucs = True AND Cardinality(l3_hilucs) = 1",
                   "l3_hilucs[1]"))

# 7 - Si la parcelle cadastrale est caractérisée par la présence d'un recypark, attribuer la classe 
# 'traitement des déchets'
list_rules.append(("recypark_count is NOT NULL","'4_3_3'")) 

# 8 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% d'une activité de 
# type 'aéroportuaire' au sens de l'IGN, attribuer la classe 'transport aérien'
list_rules.append(("aeroport_coverage > 0.5","'4_1_3'")) 

# 9 à 14 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% d'une activité 
# 'seveso' et qu'une des BD fourni un niveau thématique plus détaillé que 'production secondaire' (2), 
# alors attribuer ce niveau, sinon, attribuer la classe 'production secondaire'
list_rules.append(("seveso_coverage > 0.5 AND '2_1' = ANY(all_hilucs)","'2_1'")) 
list_rules.append(("seveso_coverage > 0.5 AND '2_2' = ANY(all_hilucs)","'2_2'")) 
list_rules.append(("seveso_coverage > 0.5 AND '2_3' = ANY(all_hilucs)","'2_3'")) 
list_rules.append(("seveso_coverage > 0.5 AND '2_4' = ANY(all_hilucs)","'2_4'")) 
list_rules.append(("seveso_coverage > 0.5 AND '2_5' = ANY(all_hilucs)","'2_5'")) 
list_rules.append(("seveso_coverage > 0.5","'2'")) 

# 15 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% d'une activité de 
# type 'établissements scolaires', attribuer la classe 'services d'enseignement' 
list_rules.append(("schools_coverage > 0.5","'3_3_2'"))

# 16 - Si la parcelle cadastrale est caractérisée par la présence d'un établissement pour ainés, attribuer 
# la classe 'usage résidentiel permanent'
list_rules.append(("etab_aines_count is NOT NULL","'3_3_3'")) 

# 17 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% par une zone reprise dans
# les couches "réserve naturelle agréée", "réserve naturelle domaniale" ou "Zones militaires"
# (pour les zones "Camp Roi Albert", "Camp Elsenborn Lager" ou "Camp Lagland"), attribuer la classe
# 'zones naturelles'
list_rules.append(("nature_conservation_7_coverage > 0.5","'7'")) 

# 18 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% par une zone reprise dans
# les couches "zone humide d'intérêt biologique (ZHIB)" ou "réserve forestière", attribuer la classe
# 'zones naturelles terrestres'
list_rules.append(("nature_conservation_71_coverage > 0.5","'7_1'")) 

# 19 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% d'une 
# activité de type 'camping', attribuer la classe 'services commerciaux'
list_rules.append(("camping_coverage > 0.5","'3_1'"))

# 20 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'parc de loisir' au sens de l'IGN, attribuer la classe 'services de loisirs' 
list_rules.append(("parc_lois_coverage > 0.5","'3_4_2'"))

# 21 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'parc animalier au sens de l'IGN, attribuer la classe 'services culturels'
list_rules.append(("parc_anim_coverage > 0.5","'3_4_1'"))

# 22 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'terrain de sport' au sens de l'IGN, attribuer la classe 'infrastructures sportives'
list_rules.append(("ter_sport_coverage > 0.5","'3_4_3'"))

# 23 - Si la parcelle cadastrale est caractérisée par une population supérieure à 0 via le RNPP, par 
# la présence d'infrastructure agricole dans n'importe quelle autre base de données, attribuer 
# la classe 'infrastructure agricole'. Correspond à la définition des logements agricoles au 
# sens de la classe '1_1_2' HILUCS. 
list_rules.append(("ms_populat_tot > 0 AND '1_1_2' = ANY(all_hilucs)","'1_1_2'"))

# 24 - Si la parcelle cadastrale est caractérisée par une population supérieure à 0 via le RNPP, par 
# la présence d'une activité  au sens de DBRIS jugée compatible avec 'usage résidentiel' (exclusion 
# des classes 'industrie des matières premières', 'industrie lourde', 'production d'énergie' 
# et 'services d'enseignement', attribuer la classe 'usage résidentiel avec d'autres usages compatibles'. 
# L'exclusion des 'services d'enseignement' au sens de DBRIS qui peuvent être jugés compatible 
# avec 'usage résidentiel' est intégrée à ce niveau afin de palier à une série d'erreurs de classification 
# observées lors du contrôle de qualité (e.g. habitation du concierge, imprécision géométrique des bases 
# de données...)
list_rules.append(("ms_populat_tot > 0 AND Cardinality(dbris_rank) = 1 AND dbris_maj NOT \
IN ('2_1', '2_2', '2_4', '3_3_2')","'5_2'"))

# 25 - Si l’ensemble des activités dbris rencontrées sont compatibles avec du 'usage résidentiel', 
# attribuer la classe 'usage résidentiel avec d'autres usages compatibles'
list_rules.append(("ms_populat_tot > 0 AND Cardinality(dbris_rank) > 1 AND NOT '2_1' = ANY(dbris_rank) \
AND NOT '2_2' = ANY(dbris_rank) AND NOT '2_4' = ANY(dbris_rank) AND NOT '3_3_2' = ANY(dbris_rank)","'5_2'"))

# 26 - Si la parcelle cadastrale est caractérisée par un recouvrement d'au moins 50% par la données 
# 'camping" et que DBRIS précise la classe 'services culturels, services de loisirs et services récréatifs',
# attribuer la valeur 'services commerciaux'. Cette règle est intégrée pour palier à la définition 
# trop large des éléments repris en classe '3_4' selon la table de correspondance DBRIS
list_rules.append(("camping_coverage > 0.5 AND dbris_maj = '3_4'","'3_1'"))

# 27 - Si la parcelle cadastrale est caractérisée par une population nulle et possède une classe au 
# sein de DBRIS, attribuer cette classe
list_rules.append(("ms_populat_tot is NULL AND dbris_maj is NOT NULL","dbris_maj")) 

# 28 - Si la parcelle cadastrale est caractérisée par une population supérieure à 0 et que sa superficie est 
# inférieure à 5000 m², attribuer la classe 'usage résidentiel permanent'
list_rules.append(("ms_populat_tot > 0 AND ST_AREA(geom) < 5000","'5_1'")) 

# 29 - Si la parcelle cadastrale est caractérisée par la présence d'une éolienne et que sa superficie est 
# inférieure à 5000 m², attribuer la classe 'production d'énergie'. La condition sur la taille est indispensable
# ici pour éviter de classer de grande parcelles agricoles avec une éolienne qui n'est pas cadastrée séparément.
list_rules.append(("eoliennes_count is NOT NULL AND ST_AREA(geom) < 5000","'2_4'")) 

# 30 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'carrière' au sens de l'IGN, attribuer la classe 'industries extractives'
list_rules.append(("carrier_ign_coverage > 0.5","'1_3'")) 

# 31 - Si la parcelle cadastrale est caractérisée par la présence d'une 'carrière' au sens de 
# l'inventaire du SPW, attribuer la classe 'industries extractives'
list_rules.append(("carrier_spw_count is NOT NULL","'1_3'"))  

# 32 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'SAR', attribuer la classe 'zones abandonnées'
list_rules.append(("sar_coverage > 0.5","'6_2'")) 

# 33 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une "plantations de sapins de Noël proprement dites" au sens de l'information du SPW, 
# attribuer la classe 'sapin de Noël'
list_rules.append(("sapin_noel_prop_1_1_1_C > 0.5","'1_1_1_C'")) 

# 34 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une "production de plants" ou de "pépinières et cultures horticoles non comestibles" 
# au sens de l'information du SPW, attribuer la classe 'sylviculture'
list_rules.append(("sapin_noel_prop_1_2 > 0.5","'1_2'"))

# 35 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'terre arable' au sens du SIGEC, attribuer la classe 'terre arable'
list_rules.append(("sigec_ta_coverage > 0.5","'1_1_1_B'")) 

# 36 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'prairie' au sens du SIGEC, attribuer la classe 'prairie'
list_rules.append(("sigec_p_coverage > 0.5","'1_1_1_A'")) 

# 37 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'terre arable' + 'prairie' au sens du SIGEC, attribuer la 
# classe 'production agricole commerciale' 
list_rules.append(("sigec_ta_coverage + sigec_p_coverage > 0.5","'1_1_1'")) 

# 38 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'foresterie', attribuer la classe 'sylviculture'
list_rules.append(("forestry_public_coverage > 0.5","'1_2'"))

# 39 - Si la parcelle cadastrale est caractérisée par la présence d'une éolienne, attribuer la classe 
# 'production d'énergies'. Cette règle vise les parcelles les parcelles de plus de 5000m2 avec un point éolienne, 
# mais n'ayant pas été classées comme surface agricoles dans les règles précédentes. 
list_rules.append(("eoliennes_count is NOT NULL","'2_4'"))

# 40 - Si la parcelle cadastrale est caractérisée par une population supérieure à 0, attribuer 
# la classe 'usage résidentiel permanent'
list_rules.append(("ms_populat_tot > 0","'5_1'")) 

# 41 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'éolienne' au sens du PICC, attribuer la classe 'production d'énergie'
list_rules.append(("picc_surface_prop_2_4 > 0.5","'2_4'")) 

# 42 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'camping' au sens du PICC, attribuer la classe 'services commerciaux' 
list_rules.append(("picc_surface_prop_3_1 > 0.5","'3_1'")) 

# 43 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'terrain de sport' au sens du PICC, attribuer la classe 'infrastructures sportives' 
list_rules.append(("picc_surface_prop_3_3_5 > 0.5","'3_3_5'")) 

# 44 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'terrain de sport' au sens du PICC, attribuer la classe 'infrastructures sportives' 
list_rules.append(("picc_surface_prop_3_4_3 > 0.5","'3_4_3'")) 

# 45 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'parc récréatif' au sens du PICC, attribuer la classe 'zones récréatives en plein air' 
list_rules.append(("picc_surface_prop_3_4_4 > 0.5","'3_4_4'")) 

# 46 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'station électrique' au sens du PICC, attribuer la classe 
# 'services de distribution d'électricité, de gaz et d'énergie thermique'
list_rules.append(("picc_surface_prop_4_3_1 > 0.5","'4_3_1'")) 

# 47 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'station d'épuration' au sens du PICC, attribuer la classe 
# 'infrastructures pour la distribution d'eau et l'assainissement'
list_rules.append(("picc_surface_prop_4_3_2 > 0.5","'4_3_2'")) 

# 48 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% 
# d'une activité de type 'parc à conteneurs' au sens du PICC, attribuer la classe 
# 'traitement des déchets'
list_rules.append(("picc_surface_prop_4_3_3 > 0.5","'4_3_3'")) 

# 49 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 
# 'agricole' au sens du PICC, attribuer la classe 'infrastructures agricoles'
list_rules.append(("picc_symbology_count_1_1_2 is NOT NULL","'1_1_2'")) 

# 50 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 'police', 
# 'pompier', 'prison', 'maison communale' ou 'administration' au sens du PICC, attribuer 
# la classe 'services d'administration publique, de défense et de sécurité sociale'
list_rules.append(("picc_symbology_count_3_3_1 is NOT NULL","'3_3_1'")) 

# 51 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 'scolaire', 
# 'scolaire fondamental', 'scolaire secondaire' ou scolaire supérieur' au sens du PICC, 
# attribuer la classe 'services d'enseignements'. Cette règle reste pertinente pour complémenter l'inventaire 
# des établissements scolaires au niveau du supérieur essentiellement
list_rules.append(("picc_symbology_count_3_3_2 is NOT NULL","'3_3_2'")) 

# 52 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 
# 'hôpital' ou 'maison de repos' au sens du PICC, attribuer la classe 'services de santé et d'action sociale'
list_rules.append(("picc_symbology_count_3_3_3 is NOT NULL","'3_3_3'")) 

# 53 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 
# 'lieu de culte' au sens du PICC, attribuer la classe 'services religieux'
list_rules.append(("picc_symbology_count_3_3_4 is NOT NULL","'3_3_4'")) 

# 54 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 
# 'culture, sport ou loisir' au sens du PICC, attribuer la classe 'services culturels, 
# services de loisirs et services récréatifs'
list_rules.append(("picc_symbology_count_3_4 is NOT NULL","'3_4'")) 

# 55 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 'station-service' 
# au sens du PICC, attribuer la classe 'transport routier'
list_rules.append(("picc_symbology_count_4_1_1 is NOT NULL","'4_1_1'")) 

# 56 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 'gare' 
# au sens du PICC, attribuer la classe 'transport ferroviaire'
list_rules.append(("picc_symbology_count_4_1_2 is NOT NULL","'4_1_2'")) 

# 57 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 'éolienne'
# au sens du PICC, attribuer la classe 'production d'énergie'
list_rules.append(("picc_symbology_count_2_4 is NOT NULL","'2_4'")) 

# 58 - Si la parcelle cadastrale est caractérisée par la présence d'une activité de type 'château d'eau' 
# ou 'station d'épuration' au sens du PICC, attribuer la classe 'infrastructures pour la 
# distribution de l'eau et l'assainissement'
list_rules.append(("picc_symbology_count_4_3_2 is NOT NULL","'4_3_2'")) 

# 59 - Si la parcelle cadastrale est caractérisée par une nature cadastrale de type 'sylviculture', 
# attribuer la classe 'sylviculture'. On fait l'hypothèse que si la nature cadastrale indique une activité
# de sylviculture, la parcelle doit être classé comme telle sans être classé en zone naturelle à la règle suivante.
list_rules.append(("nat_lu_maj = '1_2'","'1_2'")) 

# 60 - Si la parcelle cadastrale est caractérisée par un recouvrement de plus de 50% par une zone Natura2000, 
# attribuer la classe 'zones naturelles'
list_rules.append(("nature_conservation_n2000_coverage > 0.5","'7'")) 

# 61 - Si la parcelle cadastrale est caractérisée par des 'prairies' ou des 'terres arables' à moins de 
# 50% de la surface et par une nature cadastrale differente de '8_8', attribuer la classe 
# correspondante au sens de la nature cadastrale
list_rules.append(("(sigec_p_coverage < 0.5 OR sigec_ta_coverage < 0.5) AND \
nat_lu_maj is NOT NULL AND nat_lu_maj != '8_8'","nat_lu_maj")) 

# 62 - L'ensemble des parcelles non classées se voient attribuer la valeur 'usage inconnu cadastré' (6_6_A).
# Pour ces parcelles, nous ne disposons pas de suffisamment d'information que pour définir une 
# utilisation du sol principale.

## Create table with results

In [ ]:
# Name of table with classification results
classif_table = 'classification_results'

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Create table with results
decision_tree_classification(con, result_table_schema='results', result_table_name=classif_table, 
                           stats_table_schema='results', stats_table_name='capa_statistics_wall_a', 
                           list_rules=list_rules, colum_label="walousmaj", colum_leaf="rulebased_leaf",
                           grant_user=['tais','bbeaumont'])
# Close connexion to postgres database
con.close()

**Add index on capakey**

In [ ]:
# Add index on 'capakey'
con = create_pg_connexion(config_parameters)
create_index(con, 'results', classif_table, 'capakey')
create_index(con, 'results', classif_table, 'geom', is_geom=True)
create_index(con, 'results', classif_table, 'walousmaj')
con.close()

## Display

In [ ]:
# Create connexion to postgres database
con = create_pg_connexion(config_parameters)
# Display header
whereconditions = []
#whereconditions.append("nat_lu_maj IS NOT NULL")
#whereconditions.append("dbris_maj IS NOT NULL")
#whereconditions.append("ms_populat_tot IS NOT NULL")
#whereconditions.append("lc_prop_11 IS NOT NULL")
#whereconditions.append("camping_area IS NOT NULL")
#whereconditions.append("ter_sport_area IS NOT NULL")
#whereconditions.append("sapin_noel_prop_1_2 IS NOT NULL")
whereconditions.append("capakey IN ('85040B0191/00A000','85040B0167/00_000')") #Parcels on which we debated during last SC about natural protected areas

df = display_header(con, 'results', classif_table, where=' AND '.join(whereconditions), row_num=15)
# Close connexion to postgres database
con.close()
# Display dataframe
df

## Backup of Postgresql data

**Backup of table with classification results**

In [ ]:
# Create a back-up of a specific table
dump_table(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], 'results', classif_table, data['backup_classif_table'])

**Backup of the whole database**

In [ ]:
# Create a back-up of the whole database
dump_db(config_parameters['pg_host'],config_parameters['pg_dbname'],config_parameters['pg_user'],
           config_parameters['pg_password'], data['backup_db'])

## Restore Postgresql data

**Restore table with classification results**

Be carefull that, by default, the restoration will overwrite existing database object if they already exists. The overwrite parameter can be used to revert this behaviour.

## Export table with key columns for pre-agreement of Statbel for publication

In [ ]:
# Export the classification results
query_export = "SELECT t.geom, t.capakey, t.dbris_maj, t.dbris_maj_share, t.dbris_count_etab, \
array_to_string(t.dbris_rank,',') as dbris_rank, t.ms_populat_tot, t.rnpp_200m_mode, \
t.walousmaj, array_to_string(x.hilucslanduse_2,',') as hilucslanduse_2 \
FROM results.classification_results AS t \
JOIN results.cusw2018 AS x ON t.capakey = x.capakey \
JOIN public.zones_statbel AS z ON ST_Intersects(t.geom, z.geom)" 
output_shape = "/media/tais/data/WALOUS/Landuse/LU_Results/classif_lu/WalOUS_Subset_LU_PreAgreement_Statbel.gpkg"
ogr2ogr_export(output_shape, config_parameters, query=query_export, verbose=False)